In [37]:
import copy
import time
import math
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from matplotlib import pyplot as plt
from datetime import datetime

In [38]:
device = torch.device('cpu')

appa1 = pd.read_csv("noam_exports/appa1.csv")
appa1 = appa1.drop(columns='Unnamed: 0')

# CNN-LSTM Neural Network

In [39]:
class CNNLSTM(nn.Module):
    def __init__(self, in_channel):
        super(CNNLSTM, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channel, out_channels=40, kernel_size=(4, 37))
        self.lstm = nn.LSTM(15, 200)
        self.fc1 = nn.Linear(200, 150)
        self.fc2 = nn.Linear(150, 50)
        self.fc3 = nn.Linear(50,3)
        self.dropout1 = nn.Dropout(0.03)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
    
    def forward(self, x):
        x = self.conv(x)
        x = F.relu(x)
        x = x.view(840, 1, 15)
        x, (h_n, c_n) = self.lstm(x, (torch.zeros(1, 1, 200), torch.zeros(1, 1, 200)))
        x = F.relu(x)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.dropout3(x)
        return x

CNN_LSTM = CNNLSTM(1)
print(CNN_LSTM)

CNNLSTM(
  (conv): Conv2d(1, 40, kernel_size=(4, 37), stride=(1, 1))
  (lstm): LSTM(15, 200)
  (fc1): Linear(in_features=200, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
  (dropout1): Dropout(p=0.03, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
  (dropout3): Dropout(p=0.1, inplace=False)
)


In [40]:
appa1

,Time,ZnOR_1,ZnOR_2,LaFeO3_1,LaFeO3_2,WO3_1,WO3_2,ZnOR_1_heatR,ZnOR_2_heatR,LaFeO3_1_heatR,...,WO3_2_Age,sin_hour,cos_hour,sin_weekday,cos_weekday,sin_month,cos_month,sin_ordate,cos_ordate,year
0,2021-01-14 00:00:00,8.682981e+05,8.682981e+05,6.439906e+05,511907.200000,8.852432e+05,1.350952e+06,102.151667,99.416667,88.176667,...,0.0,0.000000,0.261799,0.126669,-0.888615,0.440593,0.282902,0.014115,0.009771,2021
1,2021-01-14 01:00:00,9.593192e+05,9.593192e+05,6.508791e+05,517576.116667,1.035262e+06,1.638224e+06,101.971667,99.230000,87.950000,...,3600.0,0.220297,0.141451,0.126669,-0.888615,0.440593,0.282902,0.014115,0.009771,2021
2,2021-01-14 02:00:00,9.476945e+05,9.476945e+05,6.515942e+05,515248.949153,1.000092e+06,1.579538e+06,102.008475,99.279661,87.988136,...,7200.0,0.238054,-0.108947,0.126669,-0.888615,0.440593,0.282902,0.014115,0.009771,2021
3,2021-01-14 03:00:00,9.539842e+05,9.539842e+05,6.532449e+05,514094.800000,9.996102e+05,1.577424e+06,101.996667,99.313333,88.010000,...,10800.0,0.036945,-0.259179,0.126669,-0.888615,0.440593,0.282902,0.014115,0.009771,2021
4,2021-01-14 04:00:00,9.880293e+05,9.880293e+05,6.515834e+05,512470.350000,1.024759e+06,1.604768e+06,102.061667,99.308333,87.995000,...,14400.0,-0.198130,-0.171123,0.126669,-0.888615,0.440593,0.282902,0.014115,0.009771,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13696,2022-07-25 09:00:00,4.990000e+08,4.990000e+08,2.746416e+06,896828.316667,2.570732e+07,1.694804e+08,128.053333,128.596667,500.000000,...,39085200.0,0.107892,-0.238533,0.000000,0.897598,0.343997,0.394742,-0.016226,0.005605,2022
13697,2022-07-25 10:00:00,4.990000e+08,4.990000e+08,3.817816e+06,823806.716667,3.907949e+07,2.790144e+08,128.138333,128.601667,500.000000,...,39088800.0,-0.142424,-0.219668,0.000000,0.897598,0.343997,0.394742,-0.016226,0.005605,2022
13698,2022-07-25 11:00:00,4.990000e+08,4.990000e+08,5.445447e+06,779633.583333,4.981345e+07,3.737091e+08,128.193333,128.683333,500.000000,...,39092400.0,-0.261797,0.001159,0.000000,0.897598,0.343997,0.394742,-0.016226,0.005605,2022
13699,2022-07-25 12:00:00,4.990000e+08,4.990000e+08,8.055509e+06,745091.766667,6.046691e+07,4.865077e+08,128.183333,128.688333,500.000000,...,39096000.0,-0.140474,0.220920,0.000000,0.897598,0.343997,0.394742,-0.016226,0.005605,2022


In [41]:
X = appa1[[
       'ZnOR_1', 'ZnOR_2',
       'LaFeO3_1', 'LaFeO3_2',
       'WO3_1', 'WO3_2',
       'ZnOR_1_heatR', 'ZnOR_2_heatR',
       'LaFeO3_1_heatR', 'LaFeO3_2_heatR',
       'WO3_1_heatR', 'WO3_2_heatR',
       'ZnOR_1_heatV', 'ZnOR_2_heatV',
       'LaFeO3_1_heatV', 'LaFeO3_2_heatV',
       'WO3_1_heatV', 'WO3_2_heatV',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_Age', 'ZnOR_2_Age',
       'LaFeO3_1_Age', 'LaFeO3_2_Age',
       'WO3_1_Age', 'WO3_2_Age',
       'sin_hour', 'cos_hour',
       'sin_weekday', 'cos_weekday',
       'sin_month', 'cos_month',
       'sin_ordate', 'cos_ordate',
       'year'
    ]].to_numpy()
Y = appa1[['NO2', 'O3', 'CO']].to_numpy()

In [42]:
X.shape

(13701, 37)

In [43]:
scalerX = preprocessing.StandardScaler().fit(X)
X = scalerX.transform(X)
scalerY = preprocessing.StandardScaler().fit(Y)
Y = scalerY.transform(Y)

In [44]:
data = []
for i in range(len(X)):
    data.append([X[i], Y[i]])

In [45]:
X = np.lib.stride_tricks.sliding_window_view(X, (24,37))
Y = Y[X.shape[1]-1:]
(X.shape,Y.shape)

((13678, 1, 24, 37), (13701, 3))

In [46]:
vsplit = round(len(data) * 0.2)

train_dl = DataLoader(data[vsplit:], batch_size=15, shuffle=True)
test_dl = DataLoader(data[:vsplit], batch_size=15, shuffle=True)

In [47]:
loss_hist = []
loss_val_hist = []
models = []

def train_model(train_dl, model, epochs):
    global bm, loss_hist, loss_val_hist
    model.to(device)

    best_loss = float("inf")
    # define the optimization
    criterion = nn.L1Loss()
    optimizer = torch.optim.SGD(CNN_LSTM.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in tqdm(range(epochs)):
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model ou
            yhat = model(inputs.float().to(device))
            loss = criterion(yhat, targets.float().to(device))
            # credit assignment
            optimizer.zero_grad()
            loss.backward()
            # update model weights
            optimizer.step()
            
        loss_hist.append(loss.detach().cpu().numpy().item())
        val_loss = evaluate_model(test_dl, model)[0]
        loss_val_hist.append(val_loss)
        if best_loss > loss:
            models.append([loss, val_loss, copy.deepcopy(model)])
        print(f"epoch {epoch} done, loss {loss} loss val {loss_val_hist[-1]}")

In [48]:
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in tqdm(enumerate(test_dl)):
        # evaluate the model on the test set
        yhat = model(inputs.float())
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # calculate accuracy
    acc = (actuals, predictions)
    return mean_absolute_error(actuals, predictions), (predictions, actuals)

In [49]:
train_model(train_dl, CNNLSTM, 200)

AttributeError: 'torch.device' object has no attribute '_apply'

In [ ]:
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
evals = evaluate_model(test_dl, CNNLSTM)[1]
evals[1]

array([[-0.54618671,  1.06808293, -0.41565641],
       [ 0.68755297, -0.16362373,  0.05611054],
       [-1.16305655,  1.1450646 , -1.10662821],
       ...,
       [-0.93874025,  1.42733071, -0.71587175],
       [ 1.08010651, -1.26702762,  2.34346545],
       [ 3.26719049, -1.26702762,  4.36872761]])